In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import pandas as pd
import random
from datetime import datetime, date
import datetime 
import time
import os
import csv
from concurrent.futures import ThreadPoolExecutor
import openpyxl

In [2]:
options= webdriver.ChromeOptions()
# options.add_argument('--headless')  #後臺運行
options.add_argument('--no-sandbox') 
# options.add_argument('--disable-gpu') #ban GPU加速
# options.add_argument('--disable-dev-shm-usage') #ban內存
#完成設定
driver = uc.Chrome(options=options)
driver.set_window_size(700, 1000)
driver.get('https://www.chinatimes.com/?chdtv')

In [3]:
url = []
Category = []
NEWS_category = []
news_url = []
title =  []
publish_time = []
content = []

data = {
    'Link': url,
    'Category': Category,
   }

compare_news_time = datetime.date.today().strftime('%Y/%m/%d')   #這邊改過
print(compare_news_time)
now = datetime.date.today().strftime('%Y-%m-%d')

2025/02/23


In [4]:
category_bar = driver.find_elements(By.CLASS_NAME,'main-nav-item') #這是index 列


for index in category_bar[1:21]:
    a_tag = index.find_element(By.TAG_NAME, 'a')  #這個是index的href
    time.sleep(0.01)
    Category.append(index.text)   #這個是index 內的category名
    if a_tag:
        href = a_tag.get_attribute('href')
        url.append(href)

In [5]:
def process_news_page(href):
    driver = webdriver.Chrome()  # 多線程 
    driver.get(href)
    time.sleep(5)

    try:
        news_category = driver.find_elements(By.TAG_NAME, 'span')[1].get_attribute("innerText")
        news_title = driver.find_element(By.CLASS_NAME, 'article-title').text
        news_date = driver.find_element(By.CLASS_NAME, 'date').text
        article_bodies = driver.find_elements(By.CLASS_NAME, 'article-body')
        news_content = ""
        for body in article_bodies:
            paragraphs = body.find_elements(By.TAG_NAME, 'p')
            for paragraph in paragraphs:
                news_content += paragraph.text + "\n"
        
        news_data = {
            'Link': href,
            'Category': news_category,
            'Title': news_title,
            'Publish_time': news_date,
            'Text': news_content
        }
        
        print(f"category: {news_category}")
        print(f"Title: {news_title}")
        print(f"Date: {news_date}")
        print(f"Content: {news_content}")
        
        return news_data

    except Exception as e:
        print(f"Error getting news details from {href}: {e}")
        return None
     
    finally:
        driver.quit()  


all_news_data = []


for get_url in data['Link']:
    print(get_url) 

    try:
        driver.get(get_url)
        time.sleep(1)

        try:
            more = driver.find_element(By.CLASS_NAME, 'more')  # 找 "more" 按钮
            print(more.text)
            more.click()  # 案 "more" 按钮
            time.sleep(10)  

            while True:
                time.sleep(5)
                news_pages = driver.find_elements(By.CLASS_NAME, 'articlebox-compact')  # 放新聞的box
                news_urls = []
                all_today_news = True  # 標記是否所有新文都是今天的

                for get_news in news_pages:
                    news_time = get_news.find_element(By.CLASS_NAME, 'date').text

                    if news_time == compare_news_time:
                        page_links = get_news.find_elements(By.TAG_NAME, 'h3')

                        for page_link in page_links:
                            a_tags = page_link.find_elements(By.TAG_NAME, 'a')
                            for a_tag in a_tags:
                                href = a_tag.get_attribute('href')

                                if len(href) <= 300:  # 判斷廣告
                                    news_urls.append(href)
                                    print('news link' + ': ' + href)
                    else:
                        all_today_news = False
                        break

                
                with ThreadPoolExecutor(max_workers=10) as executor:  #lag就縮減
                    
                    news_data_list = list(executor.map(process_news_page, news_urls))
                    
                    all_news_data.extend(news_data_list)

                if not all_today_news:
                    print("No more news from today on this page.")
                    break  

                try:
                    next_page = driver.find_element(By.XPATH, '//a[contains(@class, "page-link") and text()="下一頁"]')
                    next_page.click()
                    time.sleep(10)
                except Exception as e:
                    print('Cannot click next page: ')
                    break  # 

        except Exception as e:
            print(f"Could not click 'more' button on {get_url}")

    except Exception as e:
        print(f"Error accessing {get_url}")
        continue



https://www.chinatimes.com/search/%E6%9E%97%E5%B2%B1%E6%A8%BA?ctrack=pc_main_search_p02
Could not click 'more' button on https://www.chinatimes.com/search/%E6%9E%97%E5%B2%B1%E6%A8%BA?ctrack=pc_main_search_p02
https://www.chinatimes.com/realtimenews/?chdtv
Could not click 'more' button on https://www.chinatimes.com/realtimenews/?chdtv
https://www.chinatimes.com/politic/?chdtv
更多
政治 最新新聞


KeyboardInterrupt: 

In [ ]:
csv_file = 'chinatimes_' + now + '.csv'
file_path = "C:/Users/Administrator/INTERN/Chinatimes_news"
csv_file_path = os.path.join(file_path, csv_file)

with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as file:
    fieldnames = ['Link', 'Category', 'Title', 'Publish_time', 'Text']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    writer.writeheader()
    for news_data in all_news_data:
        if news_data:
            writer.writerow(news_data)

driver.quit()

In [ ]:
driver.quit()